## Use Audacity to load in .stream~ and export 6 channels of audio
## Process mono files and concatenate into 6-channel wav files
(skip if already done)

In [1]:
import soundfile as sf
import numpy as np

data1,fs=sf.read('./audio_6channels17-01.WAV')
data2,fs=sf.read('./audio_6channels17-02.WAV')
data3,fs=sf.read('./audio_6channels17-03.WAV')
data4,fs=sf.read('./audio_6channels17-04.WAV')
data5,fs=sf.read('./audio_6channels17-05.WAV')
data6,fs=sf.read('./audio_6channels17-06.WAV')



In [19]:
#data = np.zeros((data1.shape[0],6))
data = np.stack((data1,data2,data3,data4,data5,data6),axis=1)
print(data.shape)

(58907136, 6)


In [20]:
filename = 'audio17_6.wav'
sf.write(filename, data, fs)

## Overview of the audio file in question 


In [21]:
from scipy.io import wavfile
import csv

#fs, data = wavfile.read('audio17_6.wav')

readings = np.zeros([5])
max_samples = 10
ti=int(fs/5)#designate 0.2s as one block?
time=0

print('Audio file loaded')
print('Audio length:',data.shape[0]/fs,'seconds')
print('Number of blocks: ',data.shape[0] // ti)

Audio file loaded
Audio length: 3681.696 seconds
Number of blocks:  18408


## Basic setup of microphone array and pyroomacoustic

In [25]:
import pyroomacoustics as pra
from pyroomacoustics.doa import circ_dist

######
# We define a meaningful distance measure on the circle

# Location of original source
azimuth = 61. / 180. * np.pi  # 60 degrees
distance = 3.  # 3 meters

#######################
# algorithms parameters
SNR = 0.    # signal-to-noise ratio
c = 343.    # speed of sound
fs = 44100  # sampling frequency
nfft = 256  # FFT size
freq_bins = np.arange(5, 60)  # FFT bins to use for estimation

# We use a circular array with radius 15 cm # and 12 microphones
R = pra.circular_2D_array((5,5), 6, 0., 0.0463)
source = np.array([2, 5])

In [34]:
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display, HTML
import IPython

## Animate DOA at each frame, and record in txt file
total #rows written in the csv file should be #blocks = 18408, each block is 0.2 sec

In [51]:
doatxt=""
for blocks in range(0,data.shape[0] // ti):
#for blocks in range(0,10):
    signals=[]
    for i in range(0,data.shape[1]):
        signals.append(data[blocks*ti:(blocks+1)*ti,i])
    #print(len(signals[0]))
    X = np.array([ 
        pra.stft(signal, nfft, nfft // 2, transform=np.fft.rfft).T 
        for signal in signals ])
    #X shape should be (6,129,24)
    
    algo_name="SRP"   
    # Construct the new DOA object
    # the max_four parameter is necessary for FRIDA only
    doa = pra.doa.algorithms[algo_name](R, fs, nfft, c=c, max_four=4)
    
    # this call here perform localization on the frames in X
    doa.locate_sources(X, freq_bins=freq_bins)
    
    #for plotting the animation
    #doa.polar_plt_dirac()
    #plt.title(algo_name)
    #clear_output(wait=True)
    #plt.show()
    
    doatxt += str(blocks*0.2)+" "+str(doa.azimuth_recon[0] / np.pi * 180.)+"\n"
    if blocks%100==0:
        #this is the percentage of being done
        print(blocks/(data.shape[0] // ti)*100,"%")


f= open("./doa17.txt","w+")
f.write(doatxt)
f.close()


0.0 %
0.5432420686657975 %
1.086484137331595 %
1.6297262059973925 %
2.17296827466319 %
2.7162103433289873 %
3.259452411994785 %
3.8026944806605822 %
4.34593654932638 %
4.889178617992177 %
5.4324206866579745 %
5.975662755323772 %
6.51890482398957 %
7.062146892655368 %
7.6053889613211645 %
8.148631029986962 %
8.69187309865276 %
9.235115167318558 %
9.778357235984354 %
10.321599304650151 %
10.864841373315949 %
11.408083441981747 %
11.951325510647544 %
12.494567579313342 %
13.03780964797914 %
13.581051716644938 %
14.124293785310735 %
14.667535853976531 %
15.210777922642329 %
15.754019991308127 %
16.297262059973924 %
16.840504128639722 %
17.38374619730552 %
17.926988265971318 %
18.470230334637115 %
19.013472403302913 %
19.556714471968707 %
20.099956540634505 %
20.643198609300303 %
21.1864406779661 %
21.729682746631898 %
22.272924815297696 %
22.816166883963493 %
23.35940895262929 %
23.90265102129509 %
24.445893089960887 %
24.989135158626684 %
25.53237722729248 %
26.07561929595828 %
26.6188613

## with txt file, doa info at each block, this should be synchronized with rest of the recorder data